In [3]:
import os
import pandas as pd

# Question 8

You will not get anywhere with your models, if you cannot make them read clinical data…
To answer the next question, you will have to write code processing a set of HL7 messages (saved as plain text files).  The messages are located in the Clinical_HL7_Samples/ subfolders (loaded from a public HL7 test repository)


In [ ]:
In what year was the youngest male patient born?

In [2]:
# Define a function to design the desired structure to store the HL7 messages:
# {folder_name:{message_name:message}}

def make_hl7_dict(root_folder):
    
    hl7_dict = {}
    
    # Iterate through folders in root folder
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        
        # Checking if it's a directory
        if os.path.isdir(folder_path):
            folder_dict = {}
            for txt_file_name in os.listdir(folder_path):
                txt_file_path = os.path.join(folder_path, txt_file_name)
                
                # Retrieving text files exclusively
                if txt_file_name.endswith('.txt'):
                    with open(txt_file_path, 'r') as f:
                        message = f.read()
                        folder_dict[txt_file_name] = message
            hl7_dict[folder_name] = folder_dict
            
    return hl7_dict


# Using absolute path of root folder
root_folder = "/Users/jamesliounis/Documents/Harvard SM DS/NCE512/NCE_512_Problem_Sets/NCE512_PS1/Clinical_HL7_Samples"

# Invoking previously defined function on root folder 
hl7_dict = make_hl7_dict(root_folder)

# Create list to store unique messages
unique_messages = []

# Iterate through the dictionaries that store the messages
for message_dict in hl7_dict.values():
    # Iterate through  messages
    for message in message_dict.values():
        # Retrieve individual messages, splitting on new line as per Hl7 guidelines
        unique_messages.append(message.split("n"))
        

# Define a function to extract DOB from string
def extract_dob(hl7_string):
    hl7_fields = hl7_string.split("|")
    dob_field = None
    for field in hl7_fields:
        if len(field) == 8 and field.isdigit():
            dob_field = field
            break
    return dob_field


# Create list to store full DOB, from which we will retrieve YOB
DOBs = []

# Iterate through unique messages
for message in unique_messages:
    
    # Iterate through individual HL7 strings
    for hl7_string in message:
        
        # Invoke previously defined function to retrieve DOB
        DOBs.append(extract_dob(hl7_string))
        
# Extracting first 4 digits (YOB)
YOBs = [int(dob[:4]) for dob in DOBs if dob is not None]

# Retrieving largest value
YOB_of_youngest_male = max(YOBs)

print("The youngest male patient was born in", YOB_of_youngest_male)


The youngest male patient was born in 2013


# Question 9

In [5]:
PATH = "/Users/jamesliounis/Documents/Harvard SM DS/NCE512/NCE_512_Problem_Sets/NCE512_PS1/COVID_Data_Small.xlsx"

covid_df = pd.read_excel(PATH)

In [6]:
# EDA

covid_df.head()

,Arrival_Date,AgeYears,Temperature,BMI,AvgReading_Neuts_pct,Respiration_Rate,HasConnectiveTissueDisorderFLG,O2_Saturation,MaleFLG,Outcome_48Hours_Dispo,Rand
0,2020-05-28,75.833333,98.0,36.3,75.6,98,1,99,1,0,0.055920
1,2020-03-10,0.916667,102.0,34.6,71.2,70,0,98,0,1,0.112461
2,2020-05-29,61.083333,97.5,25.3,68.4,61,1,99,1,0,0.031409
3,2020-05-11,56.166667,97.9,30.3,96.5,56,0,98,1,1,0.720525
4,2020-06-19,41.916667,97.0,34.1,53.7,56,0,100,0,0,0.047407
